This notebook is based on the tutorial of the OpenMined privacy preserving AI first step Pysyft Library.  



# For a first introduction to the basics 
you can watch this video expalning the technqiues used in Privacy preserving AI which are: 
* Federated Learning 
* Secure multi party computation 
* Federated Learning 


video link: https://www.youtube.com/watch?v=39hNjnhY7cY




Fisr we install Pytorch whci is the library on top of which Pysyft works on. 

In [0]:
!pip install -q torch==1.0.0 torchvision

In [2]:
!pip install syft
!pip install numpy

     |████████████████████████████████| 215kB 5.0MB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
     |████████████████████████████████| 450kB 39.5MB/s 
     |████████████████████████████████| 389kB 40.7MB/s 
     |████████████████████████████████| 71kB 24.7MB/s 
     |████████████████████████████████| 256kB 46.7MB/s 
     |████████████████████████████████| 1.4MB 42.3MB/s 
     |████████████████████████████████| 676.9MB 21kB/s 
     |████████████████████████████████| 276kB 38.0MB/s 
     |████████████████████████████████| 51kB 17.2MB/s 
     |████████████████████████████████| 122kB 43.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing ins

A tensor is the genreal word for a variable, a vector, a matrix , a 3D array etc.. 
Pysyft uses pointers of tensors. Pointers are used to facliltate the computation on data 
that it could only be seen by the data owner.
There are different types of pointers, but these ones are for more advanced usage.

In [3]:
# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.pointers import PointerTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
hook = sy.TorchHook(torch)
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

torch.tensor([1,2,3,4,5])

W0701 19:41:20.025411 140096891082624 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0701 19:41:20.045705 140096891082624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



tensor([1, 2, 3, 4, 5])

#Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to?

Well, the answer is surprisingly simple. If you're used to working in PyTorch, then you're used to working with torch.Tensor objects like these!

In [4]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)


tensor([ 2,  4,  6,  8, 10])


Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins.

#Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some other machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using Torch tensors, we're now going to work with pointers to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:


bob = sy.VirtualWorker(hook, id="bob")



Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [0]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])



And now - let's send our tensors to Bob!!


In [0]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)



BOOM! Now Bob has two tensors! Don't believe me? Have a look for yourself!


In [8]:
bob._objects


{48296231243: tensor([1, 1, 1, 1, 1]), 82854278857: tensor([1, 2, 3, 4, 5])}



Now notice something. When we called x.send(bob) it returned a new object that we called x_ptr. This is our first pointer to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata that pointers contain.


In [9]:
x_ptr

(Wrapper)>[PointerTensor | me:93809677720 -> bob:82854278857]



(Wrapper)>[PointerTensor | me:8030612864 -> bob:14597461368]

Check out that metadata!

There are two main attributes specific to pointers:

    x_ptr.location : bob, the location, a reference to the location that the pointer is pointing to
    x_ptr.id_at_location : <random integer>, the id where the tensor is stored at location

They are printed in the format <id_at_location>@<location>

There are also other more generic attributes:

    x_ptr.id : <random integer>, the id of oui pointer tensor, it was allocated randomly
    x_ptr.owner : me, the worker which owns the pointer tensor, here it's the local worker, named "me"



In [10]:


x_ptr.location



<VirtualWorker id:bob #objects:2>

In [11]:
bob

<VirtualWorker id:bob #objects:2>

In [12]:
bob == x_ptr.location

True

In [13]:
x_ptr.id_at_location

82854278857

In [14]:
x_ptr.owner

<VirtualWorker id:me #objects:0>



You may wonder why the local worker which owns the pointer is also a VirtualWorker, although we didn't create it. Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we called hook = sy.TorchHook() and so you don't usually have to create it yourself.


In [15]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [16]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [17]:
x_ptr

(Wrapper)>[PointerTensor | me:93809677720 -> bob:82854278857]

In [18]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [19]:
y_ptr

(Wrapper)>[PointerTensor | me:69126932663 -> bob:48296231243]

In [20]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [21]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

#Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor operations on remote tensors. Fortunately, tensor pointers make this quite easy! You can just use pointers like you would normal tensors!

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [24]:
z

(Wrapper)>[PointerTensor | me:85173069576 -> bob:2716848079]



And voilà!

Behind the scenes, something very powerful happened. Instead of x and y computing an addition locally, a command was serialized and sent to Bob, who performed the computation, created a tensor z, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!


In [25]:


z.get()



tensor([2, 3, 4, 5, 6])


#Torch Functions

This API has been extended to all of Torch's operations!!!


In [26]:
x

(Wrapper)>[PointerTensor | me:805753185 -> bob:68096254417]

In [27]:
y

(Wrapper)>[PointerTensor | me:38890254908 -> bob:2495096591]

In [0]:
z = torch.add(x,y)

In [29]:
z

(Wrapper)>[PointerTensor | me:88740323388 -> bob:71356426714]


#Variables (including backpropagation!)

In [0]:


x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)



In [0]:
z = (x + y).sum()

In [32]:
z.backward()

(Wrapper)>[PointerTensor | me:65767514754 -> bob:85004049282]

In [0]:


x = x.get()



In [34]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [35]:
x.grad

tensor([1., 1., 1., 1., 1.])